In [41]:
import torch
import torch.nn as nn
import json
from NegativeClassOptimization import config, utils
import os
import shutil
import glob

In [2]:
tasks_dict = {'ONE_VS_NINE': '1v9', 'HIGH_VS_95LOW': 'high_vs_95low', 'HIGH_VS_LOOSER': 'high_vs_looser'}
task_template = {'ONE_VS_NINE': '{ag}__vs__9', 'HIGH_VS_95LOW': '{ag}_high__vs__{ag}_95low', 'HIGH_VS_LOOSER': '{ag}_high__vs__{ag}_looser'}

In [42]:
def weights_2_dict(torch_model):
    torch_state_dict = torch_model.state_dict()
    weights_dict = {'l1': torch_state_dict['module.linear_1.weight'].tolist(),
                    'l1b': torch_state_dict['module.linear_1.bias'].tolist(),
                    'l2': torch_state_dict['module.linear_2.weight'].tolist(), 
                    'l2b': torch_state_dict['module.linear_2.bias'].tolist()}
    return weights_dict

In [18]:
model_path 

'./torch_models/Frozen_MiniAbsolut_ML/high_vs_looser/seed_3/split_0/1OB1_high__vs__1OB1_looser/swa_model/data/model.pth'

In [45]:
f

<_io.TextIOWrapper name='../data/attr_to_calc_1v1.json' mode='r' encoding='UTF-8'>

In [47]:
with open('../data/attr_to_calc_1v1.json', 'r') as f:
    f_data =f.read()
    tasks_1v1 = json.loads(f_data)

In [50]:
#for 1v1
for task in tasks_1v1:

    model_path = f"./torch_models/Frozen_MiniAbsolut_ML/1v1/seed_0/split_42/{task}/swa_model/data/model.pth"
    torch_model = torch.load(model_path, map_location=torch.device('cpu'))
    weights_dict = weights_2_dict(torch_model)
    with open(f'./torch_models/Frozen_MiniAbsolut_ML/model_weights_1v1/ONE_VS_ONE__{task}__0__42__weights.json', 'w') as f:
        json.dump(weights_dict, f)
    
    data_path = glob.glob(f"./torch_models/Frozen_MiniAbsolut_ML/1v1/seed_0/split_42/{task}/*test_dataset.tsv")
    if data_path != []:
        new_path = f'../data/test_datasets_1v1/ONE_VS_ONE__{task}__0__42__test.tsv'
        shutil.copy(data_path[0], new_path)


In [ ]:
#for rest
for ag in config.ANTIGENS:
    for task in tasks_dict.keys():
        for seed_id in [0,1,2,3]:
            for split_id in [0,1,2,3,4,42]:
                model_path = f"./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed_id}/split_{split_id}/{task_template[task].format(ag=ag)}/swa_model/data/model.pth"

                try:
                    torch_model = torch.load(model_path, map_location=torch.device('cpu'))
                    weights_dict = weights_2_dict(torch_model)
                    with open(f'./torch_models/Frozen_MiniAbsolut_ML/model_weights_1v1/{task}__{ag}__{seed_id}__{split_id}__weights.json', 'w') as f:
                        json.dump(weights_dict, f)
                except FileNotFoundError:
                    pass

                #extract test data

                data_path = glob.glob(f"./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed_id}/split_{split_id}/{task_template[task].format(ag=ag)}/*test_dataset.tsv")
                if data_path != []:
                    new_path = f'../data/test_datasets/{task}__{ag}__{seed_id}__{split_id}__test.tsv'
                    shutil.copy(data_path[0], new_path)
                else:
                    pass

In [26]:
!ls ./torch_models/Frozen_MiniAbsolut_ML/model_weights | wc -l

     270


In [36]:
tasks_dict.keys()

dict_keys(['ONE_VS_NINE', 'HIGH_VS_95LOW', 'HIGH_VS_LOOSER'])

In [38]:
#put weights and calculated reveal_cancel attributions to folder
attributions_file = glob.glob("../data/deeplift_revealcancel/*")
for attribution_path in attributions_file:
    file_name = os.path.basename(attribution_path)
    task, ag, seed, split = file_name.split("__")[:-1]
    weights_path = f'./torch_models/Frozen_MiniAbsolut_ML/model_weights/{task}__{ag}__{seed}__{split}__weights.json'
    destination_dir = f'./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed}/split_{split}/{task_template[task].replace("{ag}", ag)}/attributions/reveal_cancel'
    os.makedirs(destination_dir, exist_ok=True)
    attr_rescale_p = f'./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed}/split_{split}/{task_template[task].replace("{ag}", ag)}/attributions/v2.0-2/attribution_records.json'
    with open(attr_rescale_p, 'r') as f:
        json_data = f.read()
        attr_rescale = json.loads(json_data)
    with open(attribution_path, 'r') as f:
        json_data = f.read()
        attr_rev_cancel = json.loads(json_data) 
    all_attrs_rev_cancel = []
    for  record in  attr_rescale:
        record['DeepLIFT_Reveal_cancel'] = attr_rev_cancel[record['slide']]
        all_attrs_rev_cancel.append(record)
    with open(destination_dir+"/attribution_records.json", 'w') as f:
        json.dump(all_attrs_rev_cancel, f)
    shutil.copy(weights_path, destination_dir)
    #shutil.copy(attribution_path, destination_dir + "/attribution_records.json")


In [ ]:
#put weights and calculated reveal_cancel attributions to 1v1 folders
attributions_file = glob.glob("../data/deeplift_revealcancel_1v1/*")
for attribution_path in attributions_file:
    file_name = os.path.basename(attribution_path)
    task, ag1, _, ag2,  seed, split = file_name.split("__")[:-1]
    weights_path = f'./torch_models/Frozen_MiniAbsolut_ML/model_weights_1v1/{task}__{ag}__{seed}__{split}__weights.json'
    destination_dir = f'./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed}/split_{split}/{task_template[task].replace("{ag}", ag)}/attributions/reveal_cancel'
    os.makedirs(destination_dir, exist_ok=True)
    attr_rescale_p = f'./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed}/split_{split}/{task_template[task].replace("{ag}", ag)}/attributions/v2.0-2/attribution_records.json'
    with open(attr_rescale_p, 'r') as f:
        json_data = f.read()
        attr_rescale = json.loads(json_data)
    with open(attribution_path, 'r') as f:
        json_data = f.read()
        attr_rev_cancel = json.loads(json_data) 
    all_attrs_rev_cancel = []
    for  record in  attr_rescale:
        record['DeepLIFT_Reveal_cancel'] = attr_rev_cancel[record['slide']]
        all_attrs_rev_cancel.append(record)
    with open(destination_dir+"/attribution_records.json", 'w') as f:
        json.dump(all_attrs_rev_cancel, f)
    shutil.copy(weights_path, destination_dir)
    #shutil.copy(attribution_path, destination_dir + "/attribution_records.json")

In [73]:
attributions_file = glob.glob("../data/deeplift_revealcancel_1v1/*")
for attribution_path in attributions_file:
    file_name = os.path.basename(attribution_path)
    base_name = file_name.split("__")[:-1]
    task, ag1, _, ag2,  seed, split = base_name
    destination_dir = f'./torch_models/Frozen_MiniAbsolut_ML/1v1/seed_0/split_42/{ag1}__vs__{ag2}/attributions/reveal_cancel'
    os.makedirs(destination_dir, exist_ok=True)

    attr_rescale_p = f'./torch_models/Frozen_MiniAbsolut_ML/1v1/seed_0/split_42/{ag1}__vs__{ag2}/attributions/v2.0-2/attribution_records.json'
    try:
        with open(attr_rescale_p, 'r') as f:
            json_data = f.read()
            attr_rescale = json.loads(json_data)
        with open(attribution_path, 'r') as f:
            json_data = f.read()
            attr_rev_cancel = json.loads(json_data) 
        all_attrs_rev_cancel = []
        for  record in  attr_rescale:
            record['DeepLIFT_Reveal_cancel'] = attr_rev_cancel[record['slide']]
            all_attrs_rev_cancel.append(record)
        with open(destination_dir+"/attribution_records.json", 'w') as f:
            json.dump(all_attrs_rev_cancel, f)
    except:
        print(ag1, ag2)

1WEJ 3RAJ
1FBI 3VRL
5E94 1OB1
1OB1 1NSN
1H0D 1OB1
2YPV 5E94
1FBI 1ADQ
1ADQ 1OB1
3RAJ 1NSN
1NSN 5E94
1WEJ 2YPV
1ADQ 1NSN
1WEJ 1H0D
1H0D 1NSN
1H0D 1FBI
2YPV 1H0D
1OB1 3VRL
1ADQ 1FBI
3VRL 1H0D
1OB1 1ADQ
2YPV 1WEJ
1FBI 1NSN
5E94 1FBI
3VRL 2YPV
1NSN 1WEJ
1WEJ 5E94
1OB1 1FBI
1ADQ 3VRL
3VRL 3RAJ
2YPV 3RAJ
1FBI 1OB1
1H0D 1ADQ
5E94 3VRL
5E94 3RAJ
3RAJ 1H0D
1WEJ 1OB1
2YPV 3VRL
1OB1 1H0D
2YPV 1ADQ
1ADQ 3RAJ
3VRL 1ADQ
1FBI 5E94
1NSN 1FBI
1OB1 2YPV
1OB1 1WEJ
1WEJ 1NSN
2YPV 1FBI
5E94 1WEJ
1ADQ 1H0D
1OB1 3RAJ
1H0D 2YPV
1H0D 1WEJ
1ADQ 1WEJ
1ADQ 2YPV
1FBI 1H0D
1NSN 1OB1
2YPV 1NSN
1FBI 1WEJ
1FBI 2YPV
1WEJ 1FBI
1OB1 5E94
1WEJ 1ADQ
1FBI 3RAJ
1WEJ 3VRL
2YPV 1OB1
1ADQ 5E94


In [71]:
attributions_file[1]

'../data/deeplift_revealcancel_1v1/ONE_VS_ONE__1FBI__vs__3VRL__0__42__attr.json'

In [ ]:
# data_path = glob.glob(f"./torch_models/Frozen_MiniAbsolut_ML/1v1/seed_0/split_42/{task}/*test_dataset.tsv")